<a href="https://colab.research.google.com/github/fangyuan-ksgk/LLM-eval/blob/main/SPINipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Self-play Fine-Tuning | Repo Set-up
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu11
!git clone https://github.com/uclaml/SPIN.git
%cd SPIN
!python -m pip install .
!python -m pip install flash-attn --no-build-isolation

/content/SPIN


In [ ]:
!python -m pip install .
!python -m pip install flash-attn --no-build-isolation

In [ ]:
!python spin/reformat.py --output_dir UCLA-AGI/SPIN_iter0

In [15]:
# Price for the generation process is somehow too high for me ... (Also very high for others!)
# The idea is to get the pipeline running first, and then we worry about computing power here

# One of the obvious issue with the generation result, is that the LLM here is essentially trying to write a scripts, and not roleplay...

In [51]:
# Frac-Length is the length of a fraction of data batch
!accelerate launch spin/generate.py --output_dir UCLA-AGI/SPIN_iter0/generate_toy/ --frac_len 100

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Loading checkpoint shards: 100% 3/3 [00:04<00:00,  1.54s/it]
Extracting data files: 100% 2/2 [00:00<00:00, 901.61it/s]
Generating train split: 207865 examples [00:03, 65709.14 examples/s]
Generating test split: 23110 examples [00:00, 70759.84 examples/s]
Loaded Data Size:  207865
100% 7/7 [01:23<00:00, 11.96s/it]
time elapsed: 83.75502896308899


In [49]:
!accelerate launch spin/generate.py --output_dir UCLA-AGI/SPIN_iter0/generate_toy/ --frac_len 100 --data_frac 1

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Loading checkpoint shards: 100% 3/3 [00:04<00:00,  1.54s/it]
Loaded Data Size:  207865
100% 7/7 [01:32<00:00, 13.14s/it]
time elapsed: 92.05852556228638


In [50]:
!accelerate launch spin/generate.py --output_dir UCLA-AGI/SPIN_iter0/generate_toy/ --frac_len 20 --data_frac 0 --split test

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Loading checkpoint shards: 100% 3/3 [00:04<00:00,  1.54s/it]
Loaded Data Size:  23110
100% 2/2 [00:23<00:00, 11.71s/it]
time elapsed: 23.441169261932373


In [3]:
import json

# Replace 'your_file.jsonl' with your .jsonl file path
file_path = 'UCLA-AGI/SPIN_iter0/generate_toy/loser_0_test.jsonl'

def read_jsonl(file_path):
  dataset = []
  # Open the file and read line by line
  with open(file_path, 'r') as file:
    for line in file:
      # Parse each line as a JSON object
      data = json.loads(line)
      # Now 'data' is a Python dictionary
      # print(data)
      dataset.append(data)
  return dataset

In [4]:
print('Check on the first data point from the Dataset: ')
read_jsonl(file_path)[0]

Check on the first data point from the Dataset: 


{'real': [{'role': 'user',
   'content': "Write a compelling mystery story set in a vineyard, where a seasoned detective investigates a murder with twists and turns that will keep the reader engaged until the very end. Add complex characters, multiple suspects, and red herrings to create suspense and challenge the detective's deductive reasoning. Use vivid descriptive language to paint a picture of the vineyard setting, its wine-making process, and the people who live and work there. Make sure to reveal clues and motives gradually, and create a satisfying resolution that ties up all loose ends."},
  {'role': 'assistant',
   'content': "Detective Jameson had been called to the vineyard on the outskirts of town to investigate a murder. The sun was setting, casting long shadows over the grape vines, and the air was heavy with the sweet scent of fermented grapes. The body was lying in the middle of the vineyard, surrounded by broken grape vines and a scattering of grapes. The victim's thro

### Converted to read-to-train format for Self-Play Fine-Tuning

In [6]:
!python spin/convert_data.py --output_dir UCLA-AGI/SPIN_iter0/convert_toy --input_dir UCLA-AGI/SPIN_iter0/generate_toy --num_fracs 1


200
20
Extracting data files: 100% 1/1 [00:00<00:00, 1049.36it/s]
Generating train split: 200 examples [00:00, 8738.32 examples/s]
Extracting data files: 100% 1/1 [00:00<00:00, 1676.38it/s]
Generating train split: 20 examples [00:00, 7425.52 examples/s]
200
20


In [1]:
!accelerate launch --config_file configs/multi_gpu.yaml --num_processes=1 --main_process_port 29500 spin/run_spin.py configs/config.yaml

/bin/bash: line 1: accelerate: command not found
